In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Results

modes = ['vae','classall','classred','syllall','syllred','phonall','phonred'] # Triplet!!
clfs = ['mlp','logr','knn','rf','xgboost']

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies_allpart = np.mean(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['vae', 'logr']
0.7843231320990128
['classall', 'logr']
0.666907363541615
['classred', 'logr']
0.6083159421953843
['syllall', 'logr']
0.7688975467091783
['syllred', 'logr']
0.7699151170999265
['phonall', 'logr']
0.7405638838062908
['phonred', 'logr']
0.7361418566643677


In [4]:
# Results

modes = ['vae','classall','classred','syllall','syllred','phonall','phonred'] # Triplet!!
clfs = ['logr','knn','rf','xgboost']

for c in range(len(clfs)):

    for b in range(len(modes)):

        mode = modes[b]

        print([mode,clfs[c]])

        accuracies = np.load('../../results/' + mode + '/accuracies_also_valid.npy')

        frame_sizes = ['1024']

        accuracies = np.mean(accuracies,axis=-1)
        accuracies_allpart = np.mean(accuracies,axis=-2)
        
        for a in range(len(frame_sizes)):
            print(accuracies_allpart[a,c])

['vae', 'knn']
0.7874379618459404
['classall', 'knn']
0.6979463121474817
['classred', 'knn']
0.6568468635289628
['syllall', 'knn']
0.7646462202282981
['syllred', 'knn']
0.7879064800782043
['phonall', 'knn']
0.7595002140621733
['phonred', 'knn']
0.7685762661051591
